# This is a Jupyter Notebook

In [ ]:
# Import scikit-plot
import scikitplot as sp
from scikitplot import stats

# Enable notebook info logger mode
logger = sp.sp_logging.setup_logging_for_jupyter()
sp.__version__

2025-05-07 04:15:08 INFO scikitplot 140168025580608 sp_logging.py 1247 scikitplot logging level: INFO 


'0.5.0.dev0+git.20250505.f5d58d0'

In [ ]:
# Set up logging
# import logging
# logging.basicConfig(level=logging.DEBUG)
# logger = logging.getLogger(__name__)

In [ ]:
import os
import time
from pprint import pprint

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns